In [1]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/mobilenet_v2.py
import os, re, glob
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='1'

import numpy as np
import pandas as pd
import tensorflowjs as tfjs

from keras import layers, Model, callbacks
from keras.optimizers import Adam
from keras.applications.mobilenetv2 import MobileNetV2
from sklearn.model_selection import train_test_split
from ohia.metrics import top_1_accuracy, top_3_accuracy, top_5_accuracy
from ohia.encoders import FastLabelEncoder
from ohia.utils import PlantNetGenerator

Using TensorFlow backend.


In [15]:
SEED = 0
ALPHA = 1.3 # [0.35, 0.50, 0.75, 1.0, 1.3, 1.4]
BATCH_SIZE = 64
MODEL_NAME = f'mobilenetv2-{ALPHA:0.2f}'
FILE_PATH = '/home/matt/repos/ohia.ai/data'
IMAGE_DIR = 'resized_images'

In [3]:
# get list of images and labels
file_list = glob.glob(f'{FILE_PATH}/{IMAGE_DIR}/**/*.jpg', recursive=True)
full_label_list = [re.split('/', f)[-2] for f in file_list]
full_label_ids = FastLabelEncoder().fit_transform(full_label_list)

In [4]:
# split data
train_files, valid_files, train_ids, valid_ids \
    = train_test_split(file_list, full_label_ids, test_size=0.1, random_state=SEED)

In [5]:
# create generators
n_classes = len(np.unique(full_label_list))
train_generator = PlantNetGenerator(train_files, train_ids, n_classes, BATCH_SIZE, augment=True)
valid_generator = PlantNetGenerator(valid_files, valid_ids, n_classes, BATCH_SIZE, augment=False)

In [13]:
# load pretrained ImageNet model
base_model = MobileNetV2(
    input_shape=(224,224,3),
    alpha=ALPHA,
    weights='imagenet',
    include_top=False
)

# set freeze all layers
for layer in base_model.layers:
    layer.trainable = False
    
# map ImageNet features to plants
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(2048, activation='relu')(x)
outputs = layers.Dense(n_classes, activation='softmax')(x)
    
# compile the model
model = Model(inputs=base_model.input, outputs=outputs)
model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=[top_1_accuracy, top_3_accuracy, top_5_accuracy]
)

# define callbacks
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_top_3_accuracy',
        patience=10,
        verbose=1,
        mode='max',
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_top_3_accuracy',
        factor=0.5,
        patience=3,
        verbose=1,
        mode='max',
    ),
    callbacks.ModelCheckpoint(
        monitor='val_top_3_accuracy',
        filepath=f'{FILE_PATH}/weights/{MODEL_NAME}' + '_{epoch:02d}.hdf5',
        save_best_only=True,
        save_weights_only=True,
        mode='max',
    ),
]

# train model
model.fit_generator(
    generator=train_generator,
    validation_data=valid_generator,
    callbacks=callbacks_list,
    use_multiprocessing=True,
    workers=10,
    epochs=100,    
)

Epoch 1/100
165/165 [==============================] - 49s 300ms/step - loss: 1.7069 - top_1_accuracy: 0.5251 - top_3_accuracy: 0.7274 - top_5_accuracy: 0.8113 - val_loss: 1.7791 - val_top_1_accuracy: 0.4809 - val_top_3_accuracy: 0.7240 - val_top_5_accuracy: 0.8290
Epoch 2/100
165/165 [==============================] - 40s 243ms/step - loss: 1.0550 - top_1_accuracy: 0.6884 - top_3_accuracy: 0.8734 - top_5_accuracy: 0.9308 - val_loss: 1.5323 - val_top_1_accuracy: 0.5391 - val_top_3_accuracy: 0.7769 - val_top_5_accuracy: 0.8759ss: 1.0680 - to - ETA: 1s - loss: 1.0590 - top_1_accuracy: 0.6870 - top_3_accuracy: 0.8729 - top_5_ac
Epoch 3/100
165/165 [==============================] - 38s 230ms/step - loss: 0.8826 - top_1_accuracy: 0.7425 - top_3_accuracy: 0.9036 - top_5_accuracy: 0.9506 - val_loss: 1.5453 - val_top_1_accuracy: 0.5469 - val_top_3_accuracy: 0.7839 - val_top_5_accuracy: 0.8733
Epoch 4/100
165/165 [==============================] - 40s 244ms/step - loss: 0.8000 - top_1_accuracy

/home/matt/anaconda3/envs/kaggle/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.136148). Check your callbacks.
  % delta_t_median)


165/165 [==============================] - 40s 245ms/step - loss: 0.5240 - top_1_accuracy: 0.8428 - top_3_accuracy: 0.9617 - top_5_accuracy: 0.9848 - val_loss: 1.5227 - val_top_1_accuracy: 0.5894 - val_top_3_accuracy: 0.8082 - val_top_5_accuracy: 0.8941
Epoch 10/100
165/165 [==============================] - 41s 250ms/step - loss: 0.4900 - top_1_accuracy: 0.8543 - top_3_accuracy: 0.9648 - top_5_accuracy: 0.9871 - val_loss: 1.5185 - val_top_1_accuracy: 0.5816 - val_top_3_accuracy: 0.8099 - val_top_5_accuracy: 0.8958
Epoch 11/100
 18/165 [==>...........................] - ETA: 49s - loss: 0.3965 - top_1_accuracy: 0.8924 - top_3_accuracy: 0.9766 - top_5_accuracy: 0.9905

/home/matt/anaconda3/envs/kaggle/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.116888). Check your callbacks.
  % delta_t_median)


165/165 [==============================] - 43s 262ms/step - loss: 0.4715 - top_1_accuracy: 0.8639 - top_3_accuracy: 0.9676 - top_5_accuracy: 0.9863 - val_loss: 1.5160 - val_top_1_accuracy: 0.5781 - val_top_3_accuracy: 0.8125 - val_top_5_accuracy: 0.8950

Epoch 00011: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
Epoch 12/100
165/165 [==============================] - 40s 241ms/step - loss: 0.4133 - top_1_accuracy: 0.8837 - top_3_accuracy: 0.9750 - top_5_accuracy: 0.9904 - val_loss: 1.5623 - val_top_1_accuracy: 0.5712 - val_top_3_accuracy: 0.8099 - val_top_5_accuracy: 0.8898
Epoch 13/100
165/165 [==============================] - 42s 254ms/step - loss: 0.3970 - top_1_accuracy: 0.8904 - top_3_accuracy: 0.9755 - top_5_accuracy: 0.9919 - val_loss: 1.6049 - val_top_1_accuracy: 0.5729 - val_top_3_accuracy: 0.8056 - val_top_5_accuracy: 0.8924
Epoch 14/100
165/165 [==============================] - 46s 281ms/step - loss: 0.3926 - top_1_accuracy: 0.8879 - top_3_accuracy: 0.

...

KeyboardInterrupt: 

Process ForkPoolWorker-792:
Process ForkPoolWorker-784:
Process ForkPoolWorker-800:
Process ForkPoolWorker-793:
Process ForkPoolWorker-797:
Process ForkPoolWorker-785:
Process ForkPoolWorker-779:
Process ForkPoolWorker-774:
Process ForkPoolWorker-772:
Process ForkPoolWorker-778:
Process ForkPoolWorker-768:
Process ForkPoolWorker-773:
Process ForkPoolWorker-771:
Process ForkPoolWorker-767:
Process ForkPoolWorker-762:
Process ForkPoolWorker-775:
Process ForkPoolWorker-780:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-761:
Process ForkPoolWorker-776:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-763:
Process ForkPoolWorker-782:
Process ForkPoolWorker-777:
Process ForkPoolWorker-765:
Process ForkPoolWorker-770:
Process ForkPoolWorker-764:
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/matt/anaconda3/envs/kag

Traceback (most recent call last):
KeyboardInterrupt
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
KeyboardInterrupt
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/matt/anac

  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/queues.py", lin

  File "/home/matt/repos/ohia.ai/machine_learning/ohia/utils.py", line 133, in __getitem__
    img = np.array(img, dtype=np.float)/255.
KeyboardInterrupt
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/matt/anaconda3/envs/kaggle/lib/python3.6/multiprocessing/queues.py", line 347, in put
 

In [14]:
# load best model
best_weights = glob.glob(f'{FILE_PATH}/weights/{MODEL_NAME}**')
best_weights = np.sort(best_weights)[-1]
model.load_weights(best_weights)

In [16]:
# save model
tfjs.converters.save_keras_model(model, f'{FILE_PATH}/models/{MODEL_NAME}')

In [ ]:

# alpha: 1.00,  val_top_1_accuracy: 0.5521,  val_top_3_accuracy: 0.7786,  val_top_5_accuracy: 0.8698
# alpha: 1.30,  val_top_1_accuracy: 0.5842,  val_top_3_accuracy: 0.8290,  val_top_5_accuracy: 0.8993
# alpha: 1.40,  val_top_1_accuracy: 0.6068,  val_top_3_accuracy: 0.8490,  val_top_5_accuracy: 0.9149